In [41]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import * 
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from urllib.request import urlretrieve
from owslib.wfs import WebFeatureService
from dotenv import load_dotenv
from pyspark.sql import functions as F
from sklearn.metrics import mean_absolute_error
import sklearn
from pyspark.sql import Window
from pyspark.sql.functions import monotonically_increasing_id
import re
from collections import Counter


# Create a spark session
spark = (
    SparkSession.builder.appName("analysis")
    .config("spark.sql.repl.eagerEval.enabled", True) 
    .config("spark.sql.parquet.cacheMetadata", "true")
    .config("spark.sql.session.timeZone", "Etc/UTC")
    .config("spark.driver.memory", "2g")
    .config("spark.executor.memory", "6g")
    .getOrCreate()
)

ConnectionRefusedError: [Errno 111] Connection refused

# Summary

### Introduction

**Team Members:** Aryan Shahi, Jai Bretherton, Eesha Syed, Minma Herath

The goal of this project and repository was to select the 100 most profitable merchants to onboard for the new pay in 5 installments plan.

### Datasets

**Given Data:** It was identified that the transaction dataset provided would be the primary table of interest, with all other tables then being joined to this dataset.

In [ ]:
merchantdf = spark.read.parquet("../data/curated/merchantdf.parquet/")
userdf = spark.read.parquet("../data/tables/consumer_user_details.parquet")
transactiondf = spark.read.parquet("../data/curated/transactiondf.parquet/")
consumerdf = spark.read.option("header","true").csv("../data/tables/tbl_consumer.csv", sep="|")

In [ ]:
merchantdf.limit(3)

In [ ]:
userdf.limit(3)

In [ ]:
transactiondf.limit(3)

In [ ]:
consumerdf.limit(3)

**External**: We identifed age and income of customers as likely variables of interest. Therefore, we utilised a [population based dataset](https://www.abs.gov.au/statistics/people/population/regional-population-age-and-sex/latest-release) (containing a breakdown by age) and an [income based dataset](https://data.aurin.org.au/dataset/au-govt-abs-abs-personal-income-total-income-sa2-2011-2018-sa2-2016), both of which were for SA2 codes. We therefore additionally required a dataset to [map SA2 codes to postcodes](https://www.matthewproctor.com/australian_postcodes?simple=True).

We make two key assumptions with these datasets, the first being that since a postcode can belong to multiple SA2 codes, we can estimate the income and population demographics for a postcode by taking the median value of all associated SA2 codes. Additionally, we are assuming that these median postcode statistics are representive of all customers from that postcode.

In [ ]:
populationdf = pd.read_excel("../data/tables/population.xlsx", sheet_name = 'Table 3', skiprows = 7)
incomedf = spark.read.option("header","false").csv("../data/tables/datasource-AU_Govt_ABS-UoM_AURIN_DB_3abs_personal_income_total_income_sa2_2011_2018.csv")
postcodedf = spark.read.option("header","true").csv("../data/tables/australian_postcodes.csv")

In [ ]:
populationdf.head(3)

In [ ]:
incomedf.limit(3)

In [ ]:
postcodedf.limit(3)

### Pre Processing and Outlier Analysis

Much more detail is provided in the relevant preprocessing and analysis notebooks, however some key findings:

1. 1014 transactions in the dataset had a dollar value less than a cent, so was removed as they were clearly nonsensical
2. Once the revenue band and take rate were extracted, the tag field after cleanup only contained 25 unique values, a fact which became useful when segmenting merchants
3. Population dataset was given custom age groups of under 10, adolescent (aged 10-17), young adult (18-34), middle age (35-59) and old (60+)
4. An [alternative SA2 to postcode dataset](https://www.abs.gov.au/AUSSTATS/abs@.nsf/DetailsPage/1270.0.55.006July%202011?OpenDocument) was considered, which additionally provided a ratio representing the proportion of the population for a given postcode that were located in the SA2 codes it made up. While this could have allowed for the use of a weighted average, this dataset contained far less records, and would have led to a loss of almost 1.5 million transactions.
5. Only 2018 income data was used, specifically the number of earners, median age of earners, median income and mean income for all given SA2 codes.

In total, our final dataset contained roughly 11,800,000 transactions, all with information about the merchant and consumer, as well as age and income demographics for the postcode the customer resided in. 

### Fraud Data

In [ ]:
consumerfrauddf = spark.read.parquet("../data/curated/consumerfrauddf.parquet/")
merchantfrauddf = spark.read.parquet("../data/curated/merchantfrauddf.parquet/")

In [ ]:
consumerfrauddf.limit(3)

In [ ]:
merchantfrauddf.limit(3)

We were additionally supplied with both a consumer and merchant fraud dataset. We decided that, once joined with all transactions for that merchant/user and day, any transaction with a fraud_probability greater than 1.0 (i.e all the combinations contained in the dataset) would be considered fraud and hence removed. While this likely means we removed non-fraudulent transactions, our method only removed 0.005% of transactions. Therefore, there was really no significant consequence to ensuring that we almost certainty removed all fraudulent transactions from consideration. 

Additionally, since we could be fairly sure that all remaining transactions were not fraudulent, we were able to determine distributions in order to assist with predicting future fraud. It was decided if a user or merchant has 1.5 times their standard deviation in the number of transactions or transaction amounts in a day, then they will be considered fraud. 

In [ ]:
futureuserfrauddf = spark.read.parquet("../data/curated/future_user_fraud.parquet")
futuremerchantfrauddf = spark.read.parquet("../data/curated/future_merchant_fraud.parquet")

In [ ]:
futureuserfrauddf.limit(3)

In [ ]:
futuremerchantfrauddf.limit(3)

This then gives us our final dataset to be used in modelling and thus ranking:

In [ ]:
finaldf = spark.read.parquet("../data/curated/finaldf.parquet/")
finaldf.limit(5)

### Modelling

One of our very first ideas we had for the modeling and ranking of merchants was to simply onboard the merchants that had generated the most overall revenue. 

In [ ]:
modeldf1 = finaldf.withColumn("year", date_format(col("order_datetime"), "yyyy").cast("long"))
modeldf1 = modeldf1.withColumn("month", date_format(col("order_datetime"), "MM").cast("long"))
modeldf1 = modeldf1.withColumn('profit', round(finaldf.dollar_value * (finaldf.take_rate / 100), 2))

for field in ("postcode","take_rate"):
    modeldf1 = modeldf1.withColumn(
        field,
        col(field).cast('double')
    )

modeldf1 = modeldf1.drop("order_datetime","order_id","company_name","consumer_fraud_%","merchant_fraud_%")

modeldf1 = modeldf1.groupBy("year","month","merchant_abn").agg(
    round(sum("profit"),2).alias("monthly_profit"),
)
modeldf1.limit(3)

However, we realised that a simple approach such as this would be making the rather significant assumption that once a merchant is onboarded, every single transaction made with that merchant would now be made with the BNPL technology, which clearly isn’t a very realistic assumption.
 
So to improve on this idea, we decided we would still use the overall revenue, but also consider two other features. In order to determine these other features, we reviewed surveys from [C+R research](https://www.crresearch.com/blog/buy_now_pay_later_statistics) as well as [Bankrate](https://www.bankrate.com/loans/personal-loans/buy-now-pay-later-statistics/) regarding BNPL usage statistics. Some interesting statistics we found was that only 3% of respondents owed over $2 500 currently to BNPL services, and this debt was owed across an average of 3.8 different purchased items. 

Considering the strict pay in 5 installments structure that is being imposed, we determined that there was likely to be a limit as to how much customers would spend in a single BNPL transaction. Hence we should prefer merchants who not only generated large revenue, but did so through a large number of transactions. A large number of transactions would imply that these transactions are of lesser value in general, a trait we see as desirable for the statistics above. This was the motivation behind our second feature being the number of transactions.

In [ ]:
modeldf2 = finaldf.withColumn("year", date_format(col("order_datetime"), "yyyy").cast("long"))
modeldf2 = modeldf2.withColumn("month", date_format(col("order_datetime"), "MM").cast("long"))

for field in ("postcode","take_rate"):
    modeldf2 = modeldf2.withColumn(
        field,
        col(field).cast('double')
    )

modeldf2 = modeldf2.drop("order_datetime","order_id","company_name","consumer_fraud_%","merchant_fraud_%")

modeldf2 = modeldf2.groupBy("year","month","merchant_abn").agg(
    count("dollar_value").alias("monthly_transactions"),
)

modeldf2.limit(5)

However, by similar logic, we should also prefer merchants with a wider consumer base, or a merchant that makes a large amount of revenue, from frequent, smaller value transactions that are likely to be from different customers. Additionally, we found that amongst certain predefined income ranges, the highest proportion of BNPL users came from the group with average household income between \\$50 000-\\$75 000. Combining these two ideas led to our third feature, which was based on the number of unique customers for a given merchant who resides in a postcode with median income between \\$50 000 and \\$75 000.

In [ ]:
LOWER_INCOME_THRESHOLD = 50000
UPPER_INCOME_THRESHOLD = 75000
modeldf3 = finaldf.withColumn("year", date_format(col("order_datetime"), "yyyy").cast("long"))
modeldf3 = modeldf3.withColumn("month", date_format(col("order_datetime"), "MM").cast("long"))
modeldf3 = modeldf3.filter((modeldf3.median_income > LOWER_INCOME_THRESHOLD) &
                         (modeldf3.median_income < UPPER_INCOME_THRESHOLD))
                                     
for field in ("postcode","take_rate"):
    modeldf3 = modeldf3.withColumn(
        field,
        col(field).cast('double')
    )

modeldf3 = modeldf3.drop("order_datetime","order_id","company_name","consumer_fraud_%","merchant_fraud_%")
modeldf3 = modeldf3.groupBy("year","month","merchant_abn").agg(
    countDistinct("consumer_id").alias("monthly_middle_customers"),
)
modeldf3

However, rather than just deriving these features based on the previous transactions seen in the data, we aimed to identify any potential trends in these features for merchants over time, and hence try and forecast the values of these features, for every merchant in every month of 2023. 
 
Such predictions were done through the use of a simple time series regression model, which is essentially just a typical linear regression model, but with the addition of what are called lagged variables. To take the example of our model to predict future revenue, the lagged variables in this time series regression would be the predicted revenue of the merchant for the previous certain amount of months. 

The models were built using data from February 2021 - July 2022, and predictions were made and compared to the actual values for August 2022 - October 2022 to ensure the validity of the models.

In [ ]:
dfp = spark.read.parquet("../data/curated/trans_num_predictions.parquet").toPandas()
print(mean_absolute_error(dfp["monthly_transactions"],  dfp["prediction"]))

On average, the model is incorrect by 17.27 transactions

In [ ]:
dfp = spark.read.parquet("../data/curated/profit_merch_predictions.parquet").toPandas()
print(mean_absolute_error(dfp["monthly_profit"],  dfp["prediction"]))

On average, the model is incorrect by $191.49

In [ ]:
dfp = spark.read.parquet("../data/curated/medium_customers_predictions.parquet").toPandas()
print(mean_absolute_error(dfp["monthly_middle_customers"],  dfp["prediction"]))

On average, our model is incorrect by 5.49 'middle' customers

Determining how many months to use to predict the next month's features was a difficult task. On one hand, having more lags means a smaller sample size of months to build our model on, as for example, the month of February 2021 doesn’t have any data from a previous month to use as a feature, meaning it is of no use to us. So the more lags we add, the more months we discard. 

However, more lags means our model can (to an extent) more accurately predict certain trends over time. Extended analysis (see appropriate notebooks for further analysis) determined that for a large majority of merchants, two lags was seen as being the amount which balanced these factors best.

Alongside the above curated features, we determined that merchants with a relatively large population of young and middle aged adults are more desired. This was based on a research report presented by [AFIA (pg.24)](https://afia.asn.au/files/galleries/AFIA_BNPL_Research_Report.pdf) showing the BNPL Usage over time by age group, with 25-34 year olds having the highest usage, followed closely by 18-24 year olds, then 35-44 year olds, and so on. Thus, using postcode data for each merchant, we extrapolated the population of each age group using the population dataset. We then obtained the quantile values for each age group and categorized the data into the 4 quantiles as follows:

* group 1 :  LowerQuartile
* group 2 : between LowerQuatile and Median
* group 3 : between Median and UpperQuartile
* group 4 :  UpperQuartile

This was done so comparisons between population values for a given age group would be easier, e.g. for the young adult population, having less than 022 is considered small, whereas having more than 2073 is considered very large (see notebook 4.4 to get the quantile values for each age group).

In [ ]:
merch_pop = spark.read.parquet("../data/curated/merch_pop.parquet/")
merch_pop.limit(5)

### Results

#### Initial Ranking System

Ranking System uses the following features:
- future predictions for transactions per merchant per month
- future predictions for profit per merchant per month
- future predictions for customers within revenue range \\$50k-\\$75k per month
- population (quartiles) of young adults and middle aged adults based on postcodes of consumers

In [ ]:
future_trans_sdf = spark.read.parquet("../data/curated/futuretrans.parquet/")
future_profit_sdf = spark.read.parquet("../data/curated/futureprofit.parquet/")
future_mcustomers_sdf = spark.read.parquet("../data/curated/futuremcustomers.parquet/")

Create a pandas dataframe to store points for each merchant, with total points initialized to zero.

In [ ]:
merch_points_pd = future_trans_sdf \
    .select('merchant_abn') \
    .distinct() \
    .withColumns({
        'total_points': lit(1),
        'pop_points': lit(0),
        'trans_points': lit(0),
        'profit_points': lit(0),
        'mcust_points': lit(0)
    }) \
    .orderBy("merchant_abn") \
    .toPandas()

merch_points_pd

Have 4018 distinct merchants, need to select top 100 merchants

Idea:
- all merchants have initial points 1
- the more the number of points, the higher/better the rank will be
- points given based on sorted positioning within the features and importance of the feature (for transaction, profit, and mcustomers)
- award 0.125 points to merchants having a relatively large young adult and middle age population (external research) <br>
with large meaning the populations are above the lower quartile (not group 1 - see notebook 4.4)
- each feature has a weight to assign importance, with all weights summing to 1

In [ ]:
WEIGHT_POP = 0.125
WEIGHT_TRANS = 0.375
WEIGHT_PROFIT = 0.375
WEIGHT_MCUSTOMERS = 0.125

Based on the above idea, the resulting points for each merchant are as shown: (see notebook 5 for detailed implementation)

In [ ]:
merch_points_df = spark.read.parquet("../data/curated/merch_points.parquet").sort(desc("total_points")).toPandas()
merch_points_df

Get the top 100 merchants with rankings and associated data for each merchant.

In [ ]:
# merchant data
merchants_sdf = spark.read.parquet("../data/curated/merchantdf.parquet/")

# get only the top 100 
top100_sdf = spark.createDataFrame(merch_points_df.iloc[1:101,])

# join the two
top100_merch = merchants_sdf.join(top100_sdf, "merchant_abn").orderBy(desc("total_points"))

# get ranking
top100_merch = top100_merch \
    .select("*") \
    .withColumn("rank", monotonically_increasing_id()) \
    .withColumn("rank", col("rank") + lit(1))
    
top100_merch

Observe frequency of tags

In [ ]:
tags_count = top100_merch.groupBy("tags") \
    .agg(
        count("merchant_abn").alias("count")
    ) \
    .orderBy(desc("count")) \
    .toPandas()

plt.barh(tags_count["tags"], tags_count["count"])
plt.xlabel("Frequency")

#### Segmented Ranking

Konw that we have 25 unique tags.

Assign tags to categories as follows:

**Hobbies: books, movies, music, and games**
- artist supply and craft shops
- books, periodicals, and newspapers
- digital goods: books, movies, music
- hobby, toy and game shops
- music shops - musical instruments, pianos, and sheet music

**Home and Office**
- cable, satellite, and other pay television and radio services
- computer programming , data processing, and integrated systems design services
- computers, computer peripheral equipment, and software
- equipment, tool, furniture, and appliance rent al and leasing
- furniture, home furnishings and equipment shops, and manufacturers, except appliances
- stationery, office supplies and printing and writing paper
- telecom

**Outdoors**
- bicycle shops - sales and service
- florists supplies, nursery stock, and flowers
- lawn and garden supply outlets, including nurseries
- motor vehicle supplies and new parts
- tent and awning shops

**Self-care and Fashion**
- health and beauty spas
- jewelry, watch, clock, and silverware shops
- opticians, optical goods, and eyeglasses
- shoe shops
- watch, clock, and jewelry repair shops

**Other**
- antique shops - sales, repairs, and restoration services
- art dealers and galleries
- gift, card, novelty, and souvenir shops

In [ ]:
# store first word of every tag (know that each tag begins with a unique word)
segments_dic = {
    "Hobbies": ["artist", "books", "digital", "hobby", "music"],
    "Home and Office": ["cable", "computer", "computers", "equipment", "furniture", "stationery", "telecom"],
    "Outdoor": ["bicycle", "florists", "lawn", "motor", "tent"],
    "Self-care and Fashion": ["health", "jewelry", "opticians", "shoe", "watch"],
    "Other": ["antique", "art", "gift"]
}

Need to get top 10 merchants in each segment.

Following the same idea for the ranking system and noting the defined segments above, we obtain the following results:

In [ ]:
seg_ranks_sdf = spark.read.parquet("../data/curated/seg_ranks.parquet")
seg_ranks_sdf

Observe frequency of tags for each segment

In [ ]:
seg_ranks_df = seg_ranks.toPandas()
seg_ranks_grp = seg_ranks_df.groupby(["segment"])

i = 0
for segment in segments_dic.keys():
    top10 = seg_ranks_grp.get_group(segment)
    tags_count = dict(Counter(top10["tags"]))
    print(tags_count)

    plt.figure(i)
    plt.title(segment)
    plt.barh(list(tags_count.keys()), list(tags_count.values()))
    #plt.hist(top10["tags"], orientation='horizontal')

    i += 1
